<h1>Lab 3 - Flight abstractions</h1>
<h4>Ben Gruher</h4>

In [1]:
sc

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1612725409117_0001,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<SparkContext master=yarn appName=livy-session-0>

<h4>Libraries and constants</h4>

In [4]:
sc.install_pypi_package("pandas==0.25.1") #Install pandas version 0.25.1 

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
sc.install_pypi_package('scipy')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
from pyspark.sql.functions import col
from pyspark.sql import functions as F
from pyspark.sql.functions import udf
import math
import pandas as pd
from scipy import spatial

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

No module named 'scipy'
Traceback (most recent call last):
ModuleNotFoundError: No module named 'scipy'



In [2]:
day = '2020-10-18'
nextDay = '2020-10-19'
S3InputPath = 's3://<BUCKET_NAME>/{}/*'.format(day)
nextDayS3InputPath = 's3://<BUCKET_NAME>/{}/*'.format(nextDay)
S3OutputPath = 's3://<BUCKET_NAME>/gruherb/flights/{}/'.format(day)
AIRPORT_DB_LOC = 's3://<BUCKET_NAME>/GlobalAirportDatabase.csv'

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
from pyspark.sql.types import *

airports_schema = \
    StructType([
        StructField('Code', StringType(), True),
        StructField('Name', StringType(), True),
        StructField('City', StringType(), True),
        StructField('Distance', FloatType(), True)
    ])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
flight_schema = StructType(
    [
        StructField('AircraftIcao', StringType(), False),
        StructField('Operator', StringType(), True),
        StructField('From', airports_schema, True),
        StructField('To', airports_schema, True),
        StructField('DepartedAt', TimestampType(), True),
        StructField('ArrivedAt', TimestampType(), True)
    ]
)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
observation_schema = ArrayType(StructType([
    StructField("Icao", StringType(), False),
    StructField("FSeen", TimestampType(), True),
    StructField('Alt', DoubleType(), True),
    StructField('Lat', DoubleType(), True),
    StructField('Long', DoubleType(), True),
    StructField('PosTime', TimestampType(), True),
    StructField('Spd', DoubleType(), True),
    StructField('Op', StringType(), True),
    StructField('Vsi', DoubleType(), True),
    StructField('Mil', BooleanType(), True),
    StructField('From', StringType(), True),
    StructField('To', StringType(), True),
    StructField('Gnd', BooleanType(), True),
    StructField('TAlt', DoubleType(), True),
    StructField('TTrk', DoubleType(), True),
    StructField('NearestAirport', airports_schema, True)
]))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<h4>Data import</h4>

In [13]:
dayOneObservationDf = spark.read.parquet(S3InputPath)
dayTwoObservationDf = spark.read.parquet(nextDayS3InputPath)
observationDf = dayOneObservationDf.union(dayTwoObservationDf)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
observationDf.head(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Row(Icao='A185A0', OpIcao=None, Op='HUFF PAUL E     - WALKERSVILLE, MD', Reg='N1978D', FSeen=datetime.datetime(2020, 10, 18, 18, 43, 59, 499000), TSecs=None, CMsgs=1, Alt=7800, TAlt=None, Lat=33.719788, Long=-82.713032, TTrk=None, PosTime=datetime.datetime(2020, 10, 18, 18, 43, 59, 499000), Spd=106.1, Vsi=768, From=None, To=None, Gnd=False, Mil=False), Row(Icao='A07C05', OpIcao=None, Op='COOPER GARY W     - GREENBACK, TN', Reg='N130JM', FSeen=datetime.datetime(2020, 10, 18, 18, 43, 58, 480000), TSecs=1, CMsgs=1, Alt=None, TAlt=None, Lat=34.212124, Long=-119.084387, TTrk=None, PosTime=datetime.datetime(2020, 10, 18, 18, 43, 58, 480000), Spd=3.8, Vsi=None, From=None, To=None, Gnd=True, Mil=False), Row(Icao='A65382', OpIcao=None, Op='AIR LAKE AVIATION LLC     - CAMDENTON, MO', Reg='N50665', FSeen=datetime.datetime(2020, 10, 18, 18, 43, 57, 666000), TSecs=2, CMsgs=2, Alt=2100, TAlt=None, Lat=36.794144, Long=-121.726587, TTrk=None, PosTime=datetime.datetime(2020, 10, 18, 18, 44, 0, 102000)

In [ ]:
observationDf.count()

In [14]:
# filter American registrations, drop nulls in Lat, Long, and PosTime columns
observationDf = observationDf.filter(col('Reg').startswith('N')).filter((col('Lat').isNotNull()) & (col('Long').isNotNull()) & (col('PosTime').isNotNull()))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
observationDf.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+------+--------------------+------+--------------------+-----+-----+-----+----+---------+-----------+----+--------------------+-----+----+--------------------+--------------------+-----+-----+
|  Icao|OpIcao|                  Op|   Reg|               FSeen|TSecs|CMsgs|  Alt|TAlt|      Lat|       Long|TTrk|             PosTime|  Spd| Vsi|                From|                  To|  Gnd|  Mil|
+------+------+--------------------+------+--------------------+-----+-----+-----+----+---------+-----------+----+--------------------+-----+----+--------------------+--------------------+-----+-----+
|A185A0|  null|HUFF PAUL E     -...|N1978D|2020-10-18 18:43:...| null|    1| 7800|null|33.719788| -82.713032|null|2020-10-18 18:43:...|106.1| 768|                null|                null|false|false|
|A07C05|  null|COOPER GARY W    ...|N130JM|2020-10-18 18:43:...|    1|    1| null|null|34.212124|-119.084387|null|2020-10-18 18:43:...|  3.8|null|                null|                null| true|fa

<h4>Adding closest airport</h4>

In [15]:
def cartesian(lat, long):
    lat = lat * (math.pi / 180)
    long = long * (math.pi / 180)
    R = 6371 # in km, relative to center of the earth
    X = R * math.cos(lat) * math.cos(long)
    Y = R * math.cos(lat) * math.sin(long)
    return (X, Y)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
# Filter to US airports, and also omit records with (0,0) lat/long
airport_db = spark.read.option('header', True).csv(AIRPORT_DB_LOC)
airport_db = airport_db.filter((col('Country') == 'USA') & (col('LatDeg') != 0) & (col('LongDeg') != 0))
airports = airport_db.toPandas()
airports['LatLong'] = airports.apply(lambda row: cartesian(float(row['LatDeg']), float(row['LongDeg'])), axis=1)
airports.drop(columns=['ICAO Code', 'LatDeg', 'LongDeg'], inplace=True)

# Coordinates into a BTree
airport_coords = list(airports['LatLong'])
tree = spatial.KDTree(airport_coords)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
bairports = sc.broadcast(airports)
btree = sc.broadcast(tree)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
def closest_airport(lat, long):
    def cartesian(lat, long):
        lat = lat * (math.pi / 180)
        long = long * (math.pi / 180)
        R = 6371 # in km, relative to center of the earth
        X = R * math.cos(lat) * math.cos(long)
        Y = R * math.cos(lat) * math.sin(long)
        return (X, Y)
    
    airports = bairports.value
    tree = btree.value
    
    query = tree.query([cartesian(lat, long)])
    distance = query[0].tolist()[0]
    index = query[1].tolist()[0]
    airport = airports.iloc[index]
    
    return [airport['IATA Code'], airport['Name'], airport['City'], distance]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
closest_airport_udf = udf(closest_airport, airports_schema)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
observationDf = observationDf.withColumn('NearestAirport', closest_airport_udf(col('Lat'), col('Long')))
# observationDf.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<h4>Flight processing</h4>

In [21]:
cols = ['Icao', 'PosTime', 'FSeen', 'Alt', 'Lat', 'Long', 'Spd', 'Op', 'Vsi', 'Mil', 'From', 'To', 'Gnd', 'TAlt', 'TTrk', 'NearestAirport']

observationsPerAircraft = observationDf.withColumn('combined', F.struct(*cols)).\
    groupBy('Icao').agg(F.sort_array(F.collect_list('combined'))).withColumnRenamed('sort_array(collect_list(combined), true)', 'observations')
# observationsPerAircraft.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
sortedObservationsPerAircraft = observationsPerAircraft.withColumn('sorted_observations', col('observations')).select('Icao', 'sorted_observations')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [25]:
dayOfInterestYear, dayOfInterestMonth, dayOfInterestDay = day.split('-')
def isOnDay(timestamp):
    testYear = timestamp.strftime('%Y')
    testMonth = timestamp.strftime('%m')
    testDay = timestamp.strftime('%d')
    return dayOfInterestYear == testYear and dayOfInterestMonth == testMonth and dayOfInterestDay == testDay

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [31]:
@udf(returnType=ArrayType(flight_schema))
def createFlights(obList):
    flightList = []
    if len(obList) == 0:
        return []
    
    # aircraft Icao and operator Icao should be the same for all flights with the same aircraft Icao code
    aircraftIcao = obList[0]['Icao']
    operator = obList[0]['Op']
    
    flightStarted = None 
    currentFlight = None
    
    for ob in obList:
        # initial observations - could be flight that departed on day before
        if flightStarted == None:
            # if plane is on the ground, start looking for beginning of flight
            if ob['Gnd']:
                flightStarted = False
        else:
            # flight has started
            if flightStarted == False and ob['Gnd'] == False and isOnDay(ob['PosTime']):
                flightStarted = True
                currentFlight = {'AircraftIcao': aircraftIcao, 'Operator': operator, 'From': ob['NearestAirport'], 'DepartedAt': ob['PosTime']}
            # flight has ended
            elif flightStarted == True and ob['Gnd'] == True:
                flightStarted = False
                currentFlight['To'] = ob['NearestAirport']
                currentFlight['ArrivedAt'] = ob['PosTime']
                flightList.append(currentFlight)
    
    # handle flights that do not have ending observations
#     if flightStarted == True:
#         currentFlight['To'] = None
#         currentFlight['ArrivedAt'] = None
#         flightList.append(currentFlight)
    
    return flightList

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [32]:
flightsPerAircraft = sortedObservationsPerAircraft.withColumn("flights", createFlights(col("sorted_observations")))
# flightsPerAircraft.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<h4>Creating Flight Dataframe</h4>

In [33]:
flightsDf = flightsPerAircraft.select(F.explode(col('flights')).alias('flights'))
# flightsDf.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [34]:
flightsDf = flightsDf.select(flightsDf.flights['AircraftIcao'].alias('AircraftIcao'), \
                 flightsDf.flights['Operator'].alias('Operator'), \
                 flightsDf.flights['From'].alias('From'), \
                 flightsDf.flights['To'].alias('To'), \
                 flightsDf.flights['DepartedAt'].alias('DepartedAt'), \
                 flightsDf.flights['ArrivedAt'].alias('ArrivedAt') \
                )
# flightsDf.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [35]:
flightsDf.write.parquet(S3OutputPath)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<h3>Analysis</h3>

In [6]:
flightsDf = spark.read.parquet(S3OutputPath)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<h5>How many flights were there?</h5>

In [37]:
flightsDf.count() # should be about 22k

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

22692

<h5>What were the five earliest flights–the five flights that began earliest.  Present these flights in a table including:  Operator Name, Begin Time, End Time, Departure Airport Code, Arrival Airport Code.</h5>

In [7]:
flightsDf.orderBy(col('DepartedAt')).select(col('Operator'), col('DepartedAt'), col('ArrivedAt'), col('From')['Code'], col('To')['Code']).show(5, truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------------------------------------+-----------------------+-----------------------+---------+-------+
|Operator                                         |DepartedAt             |ArrivedAt              |From.Code|To.Code|
+-------------------------------------------------+-----------------------+-----------------------+---------+-------+
|Southwest Airlines                               |2020-10-18 00:00:13.831|2020-10-18 01:51:28.892|BWI      |MCO    |
|RAINIER HELI INTERNATIONAL INC     - KIRKLAND, WA|2020-10-18 00:00:14.136|2020-10-18 00:09:43.454|LAS      |LAS    |
|SMITH COLIN B     - SAN DIEGO, CA                |2020-10-18 00:00:14.58 |2020-10-18 00:40:28.859|PSP      |SAN    |
|SkyWest Airlines                                 |2020-10-18 00:00:15.782|2020-10-18 16:30:29.174|DEN      |DEN    |
|Sprockets Excel LLC                              |2020-10-18 00:00:29.049|2020-10-18 00:20:13.614|FTW      |DAL    |
+-------------------------------------------------+-----

<h5>What were the five latest flights–the five flights that begin on 2020-10-18 and arrived latest.  Present these flights in a table with the same format as the previous question.</h5>

In [8]:
flightsDf.orderBy(col('ArrivedAt'), ascending=False).select(col('Operator'), col('DepartedAt'), col('ArrivedAt'), col('From')['Code'], col('To')['Code']).show(5, truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------------------------------------+-----------------------+-----------------------+---------+-------+
|Operator                                        |DepartedAt             |ArrivedAt              |From.Code|To.Code|
+------------------------------------------------+-----------------------+-----------------------+---------+-------+
|United Airlines                                 |2020-10-18 10:36:15.707|2020-10-19 23:56:59.491|ORD      |EWR    |
|UNIVERSITY OF NORTH DAKOTA     - GRAND FORKS, ND|2020-10-18 12:04:28.645|2020-10-19 23:55:28.224|VRB      |VRB    |
|ATLAS AIR INC     - PURCHASE, NY                |2020-10-18 21:44:43.47 |2020-10-19 23:55:15.512|LAX      |ANC    |
|BRISCOE FLYERS LLC     - DULUTH, GA             |2020-10-18 21:50:15.19 |2020-10-19 23:55:15.356|AND      |CHA    |
|Med Trans                                       |2020-10-18 05:59:58.592|2020-10-19 23:55:13.843|FAT      |MOD    |
+------------------------------------------------+--------------

<h5>List the five operators that had the most flights beginning that day.  Present the operators in a table that includes Operator name, and number of flights, sorted in descending order of number of flights. </h5>

In [41]:
flightsDf.groupBy('Operator').count().orderBy(col('count'), ascending=False).show(5, truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+-----+
|Operator          |count|
+------------------+-----+
|Southwest Airlines|1639 |
|American Airlines |1480 |
|Delta Air Lines   |969  |
|Private           |944  |
|United Airlines   |707  |
+------------------+-----+
only showing top 5 rows

<h5>Which five airports had the most flights depart from it on that day.  Present the airports in a table that includes Airport Code, and number of flights.  The table should be sorted in descending order of number of flights. </h5>

In [49]:
flightsDf.groupBy(col('From')['Code']).count().orderBy(col('count'), ascending=False).show(5, truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+-----+
|From[Code]|count|
+----------+-----+
|PHX       |1344 |
|DFW       |784  |
|DEN       |700  |
|ORD       |693  |
|SLC       |581  |
+----------+-----+
only showing top 5 rows

<h5>How many flights flew out of Seatac airport on that day?  What five operators had the most flights out of Seatac airport on that day?</h5>

In [50]:
seattleFlights = flightsDf.filter(col('From')['Code'] == 'SEA')
seattleFlights.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

438

In [51]:
seattleFlights.groupBy('Operator').count().orderBy(col('count'), ascending=False).show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------+-----+
|        Operator|count|
+----------------+-----+
| Alaska Airlines|  162|
|     Horizon Air|   60|
| Delta Air Lines|   56|
|Delta Connection|   44|
|  Virgin America|   16|
+----------------+-----+
only showing top 5 rows